In [1]:
#clear all
%reset -f

#import packages
import numpy as np
import sys
import os
import pandas as pd
import mne
import matplotlib
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'lrp'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [2]:
#setting up path/ directory
#access specific file for specific participant
def load_epochs(pp_num, root_dir, erp_path):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    filename = os.path.join(pp_directory, 'mra_p%03d_run0-epo.fif' % pp_num)

    epochs = mne.read_epochs(filename)
    #baseline for each trial
    if erp_path == 'frn':
        epochs = epochs.apply_baseline(baseline = (-1.5, -1.2))
    elif erp_path == 'lrp':
        epochs = epochs.apply_baseline(baseline = (-1.3, -1.0))
    elif erp_path == 'ern':
        epochs = epochs.apply_baseline(baseline = (-1.35, -1.05))

    return epochs, pp_directory

In [3]:
#get index of trials from behavioral data
#function will read in aligned data only
def epoch_index(pp):
    
    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\ALIGNED_learningcurve_degrees.csv')
    
    ppdata = list(df.iloc[:,pp+1])
    
    index = [i for i,v in enumerate(ppdata)]
    
    #epochs_out = epochs[index]
    numtrials = len(index)
    
    #return epochs_out, numtrials
    return index, numtrials

In [4]:
def error_index(task, errsize, pp):
    
    if task == 'rdmrot':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\rdmrot_ErrorSize_index.csv')
    elif task == 'rdmmir':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\rdmmir_ErrorSize_index.csv')
    elif task == 'rot':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\rot_ErrorSize_index.csv')
    elif task == 'mir':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\mir_ErrorSize_index.csv')
    
    ppdata = list(df.iloc[:,pp+1])

    index = [i for i,v in enumerate(ppdata) if errsize == v]

    numtrials = len(index)
    
    return index, numtrials

In [5]:
#separate function to get indexed epochs for random, since we have them before rotation or mirror
def trial_list_rdm(task, pp, index):
    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\eeg\p%03d\%s\mra_p%03d_tasktrigindex.csv' % (pp, erps, pp))
    
    #even pp numbers would experience rotation first (need random before this)
    #need to also take into account missing triggers from previous tasks
    if pp % 2 == 0:
        if task == 'rdmrot':
            #get trial after first task (trial 48 to 95, 234 to 281)
            tasksdf = df[0:96]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16150]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
            
                    
        elif task == 'rdmmir':
            #get trial after first task (trial 48 to 95)
            tasksdf = df[0:282]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16153]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
            
            
    elif pp % 2 == 1:   
        if task == 'rdmrot':
            #get trial after first five tasks (trial 234 to 281)
            tasksdf = df[0:282]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16153]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
            
            
        elif task == 'rdmmir':
            #get trial after first task (trial 48 to 95)
            tasksdf = df[0:96]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16150]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
            

    
    return(triallist)
    

In [6]:
#get matching trials from eeg data, where we account for missing triggers
def epoch_trials(task, pp, epochs, index):

    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\eeg\p%03d\%s\mra_p%03d_tasktrigindex.csv' % (pp, erps, pp))
    
    if task == 'aln':
        #get trial numbers where tasktrig is 16149 and trigidx is 1
        tasksdf = df[0:48]
        subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16149]
        triallist = []
        for i in index:
            ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
            nint = int(ndat['trigidx'])
            if nint == 1:
                trial = int(ndat['ctrial'])
                triallist.append(trial)

        epochs = epochs[triallist]
    
    #for the other tasks, the order depends on their pp number
    #even pp numbers would experience rotation first (need random before this)
    #need to also take into account missing triggers from previous tasks
    if pp % 2 == 0:
        if task == 'rot':
            #rot trials are after first two tasks (trial 96 to 185)
            tasksdf = df[0:186]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16151]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
        elif task == 'mir':
            #get trial after first task (trial 282 to 371)
            tasksdf = df[0:372]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16154]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
    elif pp % 2 == 1:
        if task == 'rot':
            #get trial after first five tasks (trial 282 to 371)
            tasksdf = df[0:372]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16151]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
        elif task == 'mir':
            #get trial after first five tasks (trial 96 to 185)
            tasksdf = df[0:186]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16154]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
    
    return(epochs)

In [7]:
#save epochs
def output_condition_epochs(pp_num, data, ppdir, condname, task):
    # Save the cleaned data :)
    out_fname = os.path.join(ppdir, 'p%03d_%s_%s-epo.fif' % (pp_num, condname, task))
    data.save(out_fname, overwrite = True)

In [8]:
#save as an evoked object
def output_condition_evoked(pp_num, data, ppdir, condname, task):
    evoked = data.average()
    out_fname = os.path.join(ppdir, 'p%03d_%s_%s-ave.fif' % (pp_num, condname, task))
    evoked.save(out_fname, overwrite = True)

In [9]:
#save as evoked objects - runs for any erp type
#each perturbation condition can have separate loops

#lowpass filter cutoff
l_cutoff = 30

#ALIGNED
#have one condition for aligned

for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    idx, numtrials = epoch_index(pp)
    #print(idx)
    aldata = epoch_trials('aln', pp, data, idx)
    #print(aldata)
    output_condition_epochs(pp, aldata, ppdir, 'small_large', 'aligned')
    #filter at 30 Hz
    aldata = aldata.filter(l_freq= None, h_freq=l_cutoff)
    #then average across trials
    output_condition_evoked(pp, aldata, ppdir, 'small_large', 'aligned')


#ROTATED AND MIRROR
conditions = ['sml', 'lrg']
conditionnames = ['smlerrors', 'lrgerrors']

tasks = ['rot', 'mir']

for task in range(0, len(tasks)):
    for pp in participants:
        data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
        for cond in range(0, len(conditions)):
            erridx, errtrials = error_index(tasks[task], conditions[cond], pp)
            if len(erridx) == 0:
                break
            pdata = epoch_trials(tasks[task], pp, data, erridx)
            output_condition_epochs(pp, pdata, ppdir, conditionnames[cond], tasks[task])
            #filter at 30 Hz
            pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
            #then average across trials
            output_condition_evoked(pp, pdata, ppdir, conditionnames[cond], tasks[task])    
    

        
#RANDOM 
#combine rdm rot and rdm mir into one per participant, but get correct indices
conditions = ['sml', 'lrg']
conditionnames = ['smlerrors', 'lrgerrors']

for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    for cond in range(0, len(conditions)):
        idx, numtrials = error_index('rdmrot', conditions[cond], pp)
        rdmrot_trials = trial_list_rdm('rdmrot', pp, idx)
        #print(rdmrot_trials)
        idx, numtrials = error_index('rdmmir', conditions[cond], pp)
        rdmmir_trials = trial_list_rdm('rdmmir', pp, idx)
        #print(rdmmir_trials)
        if cond == 0:
            rdm_sml_trials = rdmrot_trials + rdmmir_trials
            rdm_sml_trials.sort()
            pdata = data[rdm_sml_trials]
        elif cond == 1:
            rdm_lrg_trials = rdmrot_trials + rdmmir_trials
            rdm_lrg_trials.sort()
            pdata = data[rdm_lrg_trials]
        output_condition_epochs(pp, pdata, ppdir, conditionnames[cond], 'rdm')
        #filter at 30 Hz
        pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
        #then average across trials
        output_condition_evoked(pp, pdata, ppdir, conditionnames[cond], 'rdm')

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\mra_p000_run0-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\lrp\mra_p001_run0-epo.fif ...


[Parallel(n_jobs=1)]: Done 3072 out of 3072 | elapsed:    1.1s finished


    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



...

[Parallel(n_jobs=1)]: Done 2688 out of 2688 | elapsed:    0.8s finished


420 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 2688 out of 2688 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 2688 out of 2688 | elapsed:    0.8s finished


In [10]:
#get grand averages - split according to early or late conditions
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#read in evoked object
tasknames = ['rot', 'rdm', 'mir']
conditionnames = ['smlerrors', 'lrgerrors']

grand_aligned = []
grand_small = []
grand_large = []

#aligned
flist = []
evoked_list = []

for pp in participants:
    id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
    pp_directory = os.path.join(id_directory, erps)
    fname = os.path.join(pp_directory, 'p%03d_%s_%s-ave.fif' % (participants[pp], 'small_large', 'aligned'))
    evoked = mne.read_evokeds(fname)
    flist.append(evoked)
    evoked = flist[pp][0]
    evoked_list.append(evoked)
        
grand_aligned = mne.grand_average(evoked_list) 
aligned_flist = evoked_list
    
#rot and rdm and mir
for task in range(0, len(tasknames)):
    for condition in range(0, len(conditionnames)):
        flist = []
        evoked_list = []
        for pp in participants:
            id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
            pp_directory = os.path.join(id_directory, erps)
            fname = os.path.join(pp_directory, 'p%03d_%s_%s-ave.fif' % (participants[pp], conditionnames[condition], tasknames[task]))
            evoked = mne.read_evokeds(fname)
            flist.append(evoked)
            evoked = flist[pp][0]
            evoked_list.append(evoked)
        if task == 0 and condition == 0:
            grand_rot_small = mne.grand_average(evoked_list)
            small_rot_flist = evoked_list
        elif task == 0 and condition == 1:
            grand_rot_large = mne.grand_average(evoked_list)
            large_rot_flist = evoked_list
        elif task == 1 and condition == 0:
            grand_rdm_small = mne.grand_average(evoked_list)
            small_rdm_flist = evoked_list
        elif task == 1 and condition == 1:
            grand_rdm_large = mne.grand_average(evoked_list)
            large_rdm_flist = evoked_list
        elif task == 2 and condition == 0:
            grand_mir_small = mne.grand_average(evoked_list)
            small_mir_flist = evoked_list
        elif task == 2 and condition == 1:
            grand_mir_large = mne.grand_average(evoked_list)
            large_mir_flist = evoked_list
            

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\p000_small_large_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\lrp\p001_small_large_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\lrp\p002_small_large_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 m

Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p020\lrp\p020_small_large_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p021\lrp\p021_small_large_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p022\lrp\p022_small_large_aligned-ave.fif ...
   

No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p008\lrp\p008_smlerrors_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p009\lrp\p009_smlerrors_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\Mi

Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p028\lrp\p028_smlerrors_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p029\lrp\p029_smlerrors_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p030\lrp\p030_smlerrors_rot-ave.fif ...
    Found the data of

Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p016\lrp\p016_lrgerrors_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 35 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p017\lrp\p017_lrgerrors_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p018\lrp\p018_lrgerrors_rot-ave.fif ...
    Found the data of

...

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p012\lrp\p012_lrgerrors_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 42 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p013\lrp\p013_lrgerrors_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 42 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p014\lrp\p014_lrgerrors_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\p000_smlerrors_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\lrp\p001_smlerrors_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\lrp\p002_smlerrors_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        

    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p021\lrp\p021_smlerrors_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 37 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p022\lrp\p022_smlerrors_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector spec

    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p009\lrp\p009_lrgerrors_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p010\lrp\p010_lrgerrors_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector spec

        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p029\lrp\p029_lrgerrors_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p030\lrp\p030_lrgerrors_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please c

In [ ]:
#convert evokeds to data frame, so we can plot the ERPs better - runs for FRN/ERN ONLY

flists = ['aln', 'smallrot', 'largerot', 'smallrdm', 'largerdm', 'smallmir', 'largemir']

for f in flists:
    newdf = pd.DataFrame()
    for pp in participants:
        if f == 'aln':
            df = aligned_flist[pp].to_data_frame()
        elif f == 'smallrot':
            df = small_rot_flist[pp].to_data_frame()
        elif f == 'largerot':
            df = large_rot_flist[pp].to_data_frame()
        elif f == 'smallrdm':
            df = small_rdm_flist[pp].to_data_frame()
        elif f == 'largerdm':
            df = large_rdm_flist[pp].to_data_frame()
        elif f == 'smallmir':
            df = small_mir_flist[pp].to_data_frame()
        elif f == 'largemir':
            df = large_mir_flist[pp].to_data_frame()
             
        time = df['time'].tolist()
        df = df[['FCz', 'F3', 'Fz', 'F4', 'C3', 'Cz', 'C4', 'P3', 'Pz', 'P4']]
        #df = df[['C3', 'C4']]
        ppdat = []
        for timept in range(0,800):
            subdf = df.loc[timept,:].mean()
            ppdat.append(subdf)
    
        newdf['pp%03d'% pp] = ppdat
    
    newdf['time'] = time
    newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_SmallLarge_%s_%s.csv' % (f, erps))
    newdf.to_csv(newdf_filename)
        

In [ ]:
#plot FRN/ERN here using MNE functions
#channels = ['FCz', 'Fz']
channels = ['FCz', 'F3', 'Fz', 'F4', 'C3', 'Cz', 'C4', 'P3', 'Pz', 'P4']
#channels = ['C3']
#channels = ['C4']
# channels = ['Oz','O1','O2']
evokeds = dict(aligned = aligned_flist, 
               small_rot = small_rot_flist,
               large_rot = large_rot_flist, 
               small_rdm = small_rdm_flist, 
               large_rdm = large_rdm_flist,
               small_mir = small_mir_flist,
               large_mir = large_mir_flist)
#plot
mne.viz.plot_compare_evokeds(evokeds, picks=channels, combine = 'mean', legend = 'lower right', ylim=dict(eeg=[-15, 5]), ci=True)

In [ ]:
#calculate difference waves for FRN/ERN
diffconds = ['smallrot', 'largerot', 'smallrdm', 'largerdm', 'smallmir', 'largemir']
smallrot_diff = []
largerot_diff = []
smallrdm_diff = []
largerdm_diff = []
smallmir_diff = []
largemir_diff = []

for cond in range(0, len(diffconds)):
    for pp in participants:
        if cond == 0:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['small_rot'][pp]], 
                                                            weights=[-1, 1])
            smallrot_diff.append(diffevks)
        elif cond == 1:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['large_rot'][pp]], 
                                                         weights=[-1, 1])
            largerot_diff.append(diffevks)
        elif cond == 2:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['small_rdm'][pp]], 
                                                         weights=[-1, 1])
            smallrdm_diff.append(diffevks)
        elif cond == 3:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['large_rdm'][pp]], 
                                                         weights=[-1, 1])
            largerdm_diff.append(diffevks)
        elif cond == 4:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['small_mir'][pp]], 
                                                         weights=[-1, 1])
            smallmir_diff.append(diffevks)
        elif cond == 5:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['large_mir'][pp]], 
                                                         weights=[-1, 1])
            largemir_diff.append(diffevks)


In [ ]:
#saved difference waves as data frame for FRN/ERN

difflists = ['smallrot', 'largerot', 'smallrdm', 'largerdm', 'smallmir', 'largemir']

for diff in difflists:
    newdf = pd.DataFrame()
    for pp in participants:
        if diff == 'smallrot':
            df = smallrot_diff[pp].to_data_frame()
        elif diff == 'largerot':
            df = largerot_diff[pp].to_data_frame()
        elif diff == 'smallrdm':
            df = smallrdm_diff[pp].to_data_frame()
        elif diff == 'largerdm':
            df = largerdm_diff[pp].to_data_frame()
        elif diff == 'smallmir':
            df = smallmir_diff[pp].to_data_frame()
        elif diff == 'largemir':
            df = largemir_diff[pp].to_data_frame()
             
        time = df['time'].tolist()
        df = df[['FCz', 'F3', 'Fz', 'F4', 'C3', 'Cz', 'C4', 'P3', 'Pz', 'P4']]
        ppdat = []
        for timept in range(0,800):
            subdf = df.loc[timept,:].mean()
            ppdat.append(subdf)
    
        newdf['pp%03d'% pp] = ppdat
    
    newdf['time'] = time
    newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Diffwaves_DF_SmallLarge_%s_%s.csv' % (diff, erps))
    newdf.to_csv(newdf_filename)
        


In [ ]:
#plot difference waves for FRN/ERN here in MNE
mne.viz.plot_compare_evokeds({'Smallrot - Aligned':smallrot_diff, 'Largerot - Aligned':largerot_diff, 'Smallrdm - Aligned':smallrdm_diff, 'Largerdm - Aligned':largerdm_diff, 'Smallmir - Aligned':smallmir_diff, 'Largemir - Aligned':largemir_diff,}, 
                             picks=channels, show_sensors='upper right',
                             combine='mean',
                             title='Difference Wave');

In [11]:
#runs for LRP ONLY: need separate files for C3 and C4

flists = ['aln', 'smallrot', 'largerot', 'smallrdm', 'largerdm', 'smallmir', 'largemir']
channels = ['C3', 'C4']
for channel in channels:
    for f in flists:
        newdf = pd.DataFrame()
        for pp in participants:
            if f == 'aln':
                df = aligned_flist[pp].to_data_frame()
            elif f == 'smallrot':
                df = small_rot_flist[pp].to_data_frame()
            elif f == 'largerot':
                df = large_rot_flist[pp].to_data_frame()
            elif f == 'smallrdm':
                df = small_rdm_flist[pp].to_data_frame()
            elif f == 'largerdm':
                df = large_rdm_flist[pp].to_data_frame()
            elif f == 'smallmir':
                df = small_mir_flist[pp].to_data_frame()
            elif f == 'largemir':
                df = large_mir_flist[pp].to_data_frame()
             
            time = df['time'].tolist()
            df = df[[channel]]
            ppdat = []
            for timept in range(0,800):
                subdf = df.loc[timept,:].mean()
                ppdat.append(subdf)
    
            newdf['pp%03d'% pp] = ppdat
    
        newdf['time'] = time
        newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_SmallLarge_%s_%s_%s.csv' % (f, erps, channel))
        newdf.to_csv(newdf_filename)

In [12]:
# Difference waves between aligned and perturbed conditions for C3
channels = ['C3']
#channels = ['C4']
# channels = ['Oz','O1','O2']
evokeds = dict(aligned = aligned_flist, 
               small_rot = small_rot_flist,
               large_rot = large_rot_flist, 
               small_rdm = small_rdm_flist, 
               large_rdm = large_rdm_flist,
               small_mir = small_mir_flist,
               large_mir = large_mir_flist)
#plot
mne.viz.plot_compare_evokeds(evokeds, picks=channels, combine = 'mean', legend = 'lower right', ylim=dict(eeg=[-15, 5]), ci=True)

C:\Users\Raphael\AppData\Local\Temp\ipykernel_17612\4008079040.py:13: RuntimeWarning: Only 1 channel in "picks"; cannot combine by method "mean".
  mne.viz.plot_compare_evokeds(evokeds, picks=channels, combine = 'mean', legend = 'lower right', ylim=dict(eeg=[-15, 5]), ci=True)


combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"


[<Figure size 800x600 with 2 Axes>]

In [13]:
#calculate difference waves for LRP C3
diffconds = ['smallrot', 'largerot', 'smallrdm', 'largerdm', 'smallmir', 'largemir']
smallrot_diff = []
largerot_diff = []
smallrdm_diff = []
largerdm_diff = []
smallmir_diff = []
largemir_diff = []

for cond in range(0, len(diffconds)):
    for pp in participants:
        if cond == 0:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['small_rot'][pp]], 
                                                            weights=[-1, 1])
            smallrot_diff.append(diffevks)
        elif cond == 1:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['large_rot'][pp]], 
                                                         weights=[-1, 1])
            largerot_diff.append(diffevks)
        elif cond == 2:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['small_rdm'][pp]], 
                                                         weights=[-1, 1])
            smallrdm_diff.append(diffevks)
        elif cond == 3:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['large_rdm'][pp]], 
                                                         weights=[-1, 1])
            largerdm_diff.append(diffevks)
        elif cond == 4:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['small_mir'][pp]], 
                                                         weights=[-1, 1])
            smallmir_diff.append(diffevks)
        elif cond == 5:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['large_mir'][pp]], 
                                                         weights=[-1, 1])
            largemir_diff.append(diffevks)

In [14]:
#saved difference waves as data frame for LRP C3

difflists = ['smallrot', 'largerot', 'smallrdm', 'largerdm', 'smallmir', 'largemir']

for diff in difflists:
    newdf = pd.DataFrame()
    for pp in participants:
        if diff == 'smallrot':
            df = smallrot_diff[pp].to_data_frame()
        elif diff == 'largerot':
            df = largerot_diff[pp].to_data_frame()
        elif diff == 'smallrdm':
            df = smallrdm_diff[pp].to_data_frame()
        elif diff == 'largerdm':
            df = largerdm_diff[pp].to_data_frame()
        elif diff == 'smallmir':
            df = smallmir_diff[pp].to_data_frame()
        elif diff == 'largemir':
            df = largemir_diff[pp].to_data_frame()
             
        time = df['time'].tolist()
        #df = df[['FCz', 'F3', 'Fz', 'F4', 'C3', 'Cz', 'C4', 'P3', 'Pz', 'P4']]
        df = df[['C3']]
        ppdat = []
        for timept in range(0,800):
            subdf = df.loc[timept,:].mean()
            ppdat.append(subdf)
    
        newdf['pp%03d'% pp] = ppdat
    
    newdf['time'] = time
    newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Diffwaves_DF_SmallLarge_%s_%s_C3.csv' % (diff, erps))
    newdf.to_csv(newdf_filename)
        

In [15]:
#plot difference waves for LRP C3 here in MNE
mne.viz.plot_compare_evokeds({'Smallrot - Aligned':smallrot_diff, 'Largerot - Aligned':largerot_diff, 'Smallrdm - Aligned':smallrdm_diff, 'Largerdm - Aligned':largerdm_diff, 'Smallmir - Aligned':smallmir_diff, 'Largemir - Aligned':largemir_diff}, 
                             picks=channels, show_sensors='upper right',
                             combine='mean',
                             title='Difference Wave');

combining channels using "mean"
combining channels using "mean"


C:\Users\Raphael\AppData\Local\Temp\ipykernel_17612\3252030966.py:2: RuntimeWarning: Only 1 channel in "picks"; cannot combine by method "mean".
  mne.viz.plot_compare_evokeds({'Smallrot - Aligned':smallrot_diff, 'Largerot - Aligned':largerot_diff, 'Smallrdm - Aligned':smallrdm_diff, 'Largerdm - Aligned':largerdm_diff, 'Smallmir - Aligned':smallmir_diff, 'Largemir - Aligned':largemir_diff},


combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
